Cyna Shirazinejad, 1/13/22

# outline of notebook 7:

* load data from cell lines:
* * AP2-tagRFP-T, tagGFP2-DNM2, ARPC3-HaloTag 
* * AP2-tagRFP-T, tagGFP2-DNM2, N-WASP-HaloTag 
* extract features from tracks
* * use existing feature scaler, decomposition axes, and mixture model to predict the identity of each new event
* merge the new data with existing tracks, features, and model cluster identities

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import pandas as pd
import sys
analysis_metadata = np.load('analysis_metadata.npy', allow_pickle=True)
sys.path.append(analysis_metadata.item().get('path_notebook')+'/cmeAnalysisPostProcessingSimplified') # add custom Python scripts to the local path
import import_tracks
import display_tracks
import detect_peaks

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
analysis_metadata

array({'path_outputs': '/Volumes/GoogleDrive/My Drive/Drubin Lab/ap2dynm2arcp3_project/stable_outputs_20220420_nwasp', 'path_notebook': '/Users/cynashirazinejad/Documents/GitHub/Jin_Shirazinejad_et_al_branched_actin_manuscript/analysis/simplified_workflow_20220420_nwasp', 'feature_units': ['seconds', 'a.u. fluorescence', 'a.u. fluorescence', 'pixels', 'pixels', 'pixels', 'pixels', 'pixels', 'seconds', 'seconds', 'seconds', 'seconds', 'seconds', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence', 'unitless', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence**2', 'a.u. fluorescence**2', 'unitless', 'unitless', 'unitless', 'unitless', 'counts', 'counts', 'unitless', 'unitless', 'unitless'], 'possible_track_features': ['lifetime', 'max_int_ch0', 'max_int_ch1', 'dist_traveled_ch0', 'dist_traveled_ch1', 'max_dist_between_ch0-ch1', 'md_ch0', 'md_ch1', 'time_to_peak_ch0', 'time_to_peak_ch1', 'time_after_peak_ch0', 'time_after_peak

In [7]:
display_tracks.upload_tracks_and_metadata?

Signature:
display_tracks.upload_tracks_and_metadata(
    path_tracks,
    path_outputs,
    analysis_metadata,
    track_categories,
    identifier_string,
    features,
    labels,
    dataframe_name,
    track_name,
    experiment_number_adjustment=0,
)
Docstring:
Format tracks contained in folders into a dataframe of extracted physical features 
as well as accompanying experimental metadata.

Args:
    analysis_metadata (dictionary): contains path to the folder containing the enclosed tracking files and output files
    track_categories (list): a list of integers for the cmeAnalysis track categories to keep for further analysis
    identifier_string (string): a label within all tracking head folders that uniquely identify relevant content
    features (list): a list with string elements containing the features to be extracted from each track
    labels (list): a list of string elements describing each features' designation in the dataframe output
    experiment_number_adjustment (i

In [8]:
possible_track_features = analysis_metadata.item().get('possible_track_features')
possible_track_features_labels = analysis_metadata.item().get('possible_track_features_labels')

In [9]:
analysis_metadata

array({'path_outputs': '/Volumes/GoogleDrive/My Drive/Drubin Lab/ap2dynm2arcp3_project/stable_outputs_20220420_nwasp', 'path_notebook': '/Users/cynashirazinejad/Documents/GitHub/Jin_Shirazinejad_et_al_branched_actin_manuscript/analysis/simplified_workflow_20220420_nwasp', 'feature_units': ['seconds', 'a.u. fluorescence', 'a.u. fluorescence', 'pixels', 'pixels', 'pixels', 'pixels', 'pixels', 'seconds', 'seconds', 'seconds', 'seconds', 'seconds', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence', 'unitless', 'a.u. fluorescence', 'a.u. fluorescence', 'a.u. fluorescence**2', 'a.u. fluorescence**2', 'unitless', 'unitless', 'unitless', 'unitless', 'counts', 'counts', 'unitless', 'unitless', 'unitless'], 'possible_track_features': ['lifetime', 'max_int_ch0', 'max_int_ch1', 'dist_traveled_ch0', 'dist_traveled_ch1', 'max_dist_between_ch0-ch1', 'md_ch0', 'md_ch1', 'time_to_peak_ch0', 'time_to_peak_ch1', 'time_after_peak_ch0', 'time_after_peak

# 3 color cell line (including arpc3 tag in addition to ap2/dnm2): upload tracks, extract features, find clusters and ccp's

In [12]:
path_adw_tracks = '/Volumes/GoogleDrive/My Drive/Drubin Lab/ap2dynm2arcp3_project/revision_tracking/adw_ap2dnm2'

In [13]:
ada_wildtype_group = {}
ada_wildtype_group['path'] = path_adw_tracks
ada_wildtype_group['df'] = 'df_adw_wildtype_merged_features'
ada_wildtype_group['tracks'] = 'merged_adw_wildtype_valid_tracks'
analysis_metadata.item()['experiment_groups']['adw_wildtype'] = ada_wildtype_group
np.save('analysis_metadata', analysis_metadata)

In [14]:
import_tracks.upload_tracks_and_metadata(analysis_metadata,
                                         'adw_wildtype',
                                         [1],
                                         ['Cell'],
                                         possible_track_features,
                                         possible_track_features_labels)


folders to mine:
71_180803_ap2-dnm2-nwasp_ap2-dnm2_wildtype_no-treatment_Cell1_1s
72_180803_ap2-dnm2-nwasp_ap2-dnm2_wildtype_no-treatment_Cell2_1s
73_180803_ap2-dnm2-nwasp_ap2-dnm2_wildtype_no-treatment_Cell3_1s
74_180803_ap2-dnm2-nwasp_ap2-dnm2_wildtype_no-treatment_Cell4_1s
75_180803_ap2-dnm2-nwasp_ap2-dnm2_wildtype_no-treatment_Cell5_1s
76_180803_ap2-dnm2-nwasp_ap2-dnm2_wildtype_no-treatment_Cell6_1s
77_180718_ap2-dnm2-nwasp_ap2-dnm2_wildtype_no-treatment_Cell1_1s
78_180718_ap2-dnm2-nwasp_ap2-dnm2_wildtype_no-treatment_Cell2_1s
79_180718_ap2-dnm2-nwasp_ap2-dnm2_wildtype_no-treatment_Cell3_1s


uploading and saving tracks...


finished uploading tracks

extracting features...

completed feature extraction

creating dataframe...

saving dataframe...

saving tracks...

done


In [15]:
display_tracks.cluster_with_existing_model(analysis_metadata,
                            'adw_wildtype')

saving dataframe...

done



,lifetime,max_int_ap2,max_int_dnm2,dist_traveled_ap2,dist_traveled_dnm2,max_dist_between_ap2_dnm2,md_ap2,md_dnm2,time_to_peak_ap2,time_to_peak_dnm2,...,number_of_tags,cell_line_tags,current_tracked_channels,experiment_type,cell_condition,framerate,date,PC-0,PC-1,gmm_predictions
0,181.0,838.846885,289.035166,7.221133,11.919019,4.888317,0.378804,0.700272,134.0,103.0,...,2.0,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180803,10.257546,-4.124278,0
1,173.0,648.969353,447.208676,3.812234,5.007210,4.533374,0.197433,0.550535,126.0,170.0,...,2.0,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180803,10.093642,-5.948446,0
2,172.0,456.305535,375.334291,19.447658,16.506035,4.124215,0.281681,0.709564,134.0,156.0,...,2.0,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180803,10.360305,-5.555795,0
3,166.0,426.657333,257.105949,14.902804,11.834867,3.237402,0.346493,0.676118,113.0,164.0,...,2.0,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180803,9.834211,-5.564749,0
4,161.0,125.206766,194.809135,8.032639,9.648622,3.170784,0.489029,1.004908,127.0,52.0,...,2.0,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180803,8.362865,-2.452186,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64530,2.0,11.723456,28.719823,4.461714,0.144131,3.998561,1.616956,1.483679,6.0,1.0,...,2.0,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180718,2.827944,5.853904,2
64531,2.0,8.436997,8.408241,6.252783,4.063372,4.560247,2.571898,2.073924,6.0,0.0,...,2.0,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180718,-8.170420,8.169081,3
64532,2.0,8.650476,19.053948,5.215851,4.816542,3.778345,1.619540,1.374823,6.0,11.0,...,2.0,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180718,1.333564,4.395938,2
64533,2.0,9.638987,4.503589,7.578272,6.815918,3.883401,1.684860,1.374672,5.0,3.0,...,2.0,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180718,-7.552481,3.406055,3


In [16]:
detect_peaks.find_single_peaks_using_existing_model(analysis_metadata,
                            'adw_wildtype')

saving dataframe...

done



,lifetime,max_int_ap2,max_int_dnm2,dist_traveled_ap2,dist_traveled_dnm2,max_dist_between_ap2_dnm2,md_ap2,md_dnm2,time_to_peak_ap2,time_to_peak_dnm2,...,cell_line_tags,current_tracked_channels,experiment_type,cell_condition,framerate,date,PC-0,PC-1,gmm_predictions,ccp_status
0,181.0,838.846885,289.035166,7.221133,11.919019,4.888317,0.378804,0.700272,134.0,103.0,...,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180803,10.257546,-4.124278,0,0.0
1,173.0,648.969353,447.208676,3.812234,5.007210,4.533374,0.197433,0.550535,126.0,170.0,...,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180803,10.093642,-5.948446,0,0.0
2,172.0,456.305535,375.334291,19.447658,16.506035,4.124215,0.281681,0.709564,134.0,156.0,...,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180803,10.360305,-5.555795,0,1.0
3,166.0,426.657333,257.105949,14.902804,11.834867,3.237402,0.346493,0.676118,113.0,164.0,...,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180803,9.834211,-5.564749,0,0.0
4,161.0,125.206766,194.809135,8.032639,9.648622,3.170784,0.489029,1.004908,127.0,52.0,...,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180803,8.362865,-2.452186,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64530,2.0,11.723456,28.719823,4.461714,0.144131,3.998561,1.616956,1.483679,6.0,1.0,...,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180718,2.827944,5.853904,2,0.0
64531,2.0,8.436997,8.408241,6.252783,4.063372,4.560247,2.571898,2.073924,6.0,0.0,...,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180718,-8.170420,8.169081,3,0.0
64532,2.0,8.650476,19.053948,5.215851,4.816542,3.778345,1.619540,1.374823,6.0,11.0,...,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180718,1.333564,4.395938,2,0.0
64533,2.0,9.638987,4.503589,7.578272,6.815918,3.883401,1.684860,1.374672,5.0,3.0,...,ap2-dnm2-nwasp,ap2-dnm2,wildtype,no-treatment,1s,180718,-7.552481,3.406055,3,0.0


<!-- # upload hypotonic shock data -->